<a href="https://colab.research.google.com/github/Firojpaudel/Machine-Learning-Notes/blob/main/Practical%20Deep%20Learning%20For%20Coders/Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MultiLabel Classification And Regression


In the last chapter, we learned some practical tips for training models, like choosing the right learning rates and number of epochs which were really important for good results. Now, this chapter dives into two new types of computer vision problems: multi-label classification (predicting multiple labels for an image or none at all) and regression (predicting numbers instead of categories). Along the way, we'll explore output activations, targets, and loss functions in more depth.

## MultiLabel Classification

*MultiLabel Classification* refers to the problem of identifying the objects in images that may not contain exactly one type of object.

There may be more than one type of object or there may be none.

In previous bear classifier we built, it had no ability to predict "not bear at all". So, this time we are trying to fix that.

First, let's see what a mutilabel dataset looks like. But before that lets set up the notebook.

In [1]:
### Setting up the notebook
##@ Notebook initialization
%reload_ext autoreload
%autoreload 2
%matplotlib inline

##@ Installing dependencies
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

##@ Importing the necessary libraries
from fastbook import *
from fastai.callback.fp16 import *
from fastai.vision.all import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Mounted at /content/gdrive


### Getting Data

We will get the PASCAL datset which has more than one kind of classified object per image.

In [ ]:
##@ Getting the PASCAL dataset
path = untar_data(URLs.PASCAL_2007)
path.ls()

<details>
<summary><b>More about this dataset:</b></summary>
This dataset is a bit different from the ones we've worked with before. Instead of being organized by filenames or folders, it comes with a CSV file that tells us which labels to use for each image.
</details>


In [ ]:
##@ inspecting the dataset
df = pd.read_csv(path/'train.csv')
df.head()

In [ ]:
header = """
===============================================================
                Pandas and DataFrames Subsection in Book
===============================================================
"""
print(header)

# Accessing rows and columns using .iloc
print("------- Using .iloc: -------")
print(f"Returning the 1st column of the dataframe:\n\n{df.iloc[:, 0]} \n")  # 1st column (labels)
print(f"Returning the 1st row of the dataframe:\n\n{df.iloc[0, :]} \n")    # 1st row (details of item1)

# Grabbing a column by name (without using iloc)
print(f"Grabbing the column 'labels' directly from the dataframe:\n\n{df['labels']}\n\n")

# Calculations in the dataframe
print("----- Calculations in the DataFrame itself ------ \n")
print("Creating a new dataframe for this one:\n")
df1 = pd.DataFrame({
    'Items Sold': [5, 2, 5, 4, 9],
    'Rates': [100, 50, 60, 400, 1000]
})
print(f"The new dataframe before the calculation:\n\n{df1}\n")
df1['Income'] = df1['Items Sold'] * df1['Rates']  # Calculating income as Items Sold * Rates

print(f"The new dataframe after the calculation:\n\n{df1}\n")

### Constriucting a DataBlock

#### Key Concepts

Before diving into the DataBlock construction, it is important to understand the two main classes used in PyTorch and FastAI to represent datasets and batches:

- **Dataset**: A collection that returns a tuple of the independent variable (input) and the dependent variable (target) for a single item.
- **DataLoader**: An iterator that provides a stream of mini-batches. Each mini-batch consists of a batch of inputs and a batch of targets.

FastAI adds two additional layers on top of these:

- **Datasets**: An iterator that contains both the training and validation datasets.
- **DataLoaders**: An object that contains a training DataLoader and a validation DataLoader.

A DataLoader is built on top of a Dataset to add functionalities like batching, shuffling, and parallel data loading.

#### Step-by-Step Construction of a DataBlock

##### **1. Starting Simple: DataBlock With No Parameters**

---
We begin my creating an empty ```DataBlock``` object with no parameters. This serves as the foundation for building the DataBlock step by step.

In [ ]:
dblock = DataBlock()

Next, we create a ```Datasets``` object from this ```DataBlock```. The only required argument is the data source—in this case, a **DataFrame**.

In [ ]:
dsets = dblock.datasets(df)

This creates a training and validation dataset, which can be accessed as:

In [ ]:
dsets.train[0]

By default, the ```DataBlock``` assumes there are two items: the input (independent variable) and the target (dependent variable). This will simply return a row of the DataFrame.

##### **2. Specifying the Input Target with ```get_x``` and ```get_y```**

---

To handle the DataFrame more effectively, we specify which columns correspond to the input and target variables using ```get_x``` and ```get_y``` functions.

In [ ]:
dblock = DataBlock(get_x = lambda r: r['fname'], get_y = lambda r: r['labels'])
dsets = dblock.datasets(df)
dsets.train[0]

In [ ]:
#@ Alternatively, we can define the functions more explicitly:
def get_x(r):
  return r['fname']

def get_y(r):
  return r['labels']

dblock = DataBlock(get_x = get_x, get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

Although using **lambda functions** is convenient, they are not compatible with serialization. For saving models after training, it's better to use the more verbose function definitions.

##### **3. Adjusting the ```get_x``` and ```get_y``` Functions:**

---

Moreover, we need convert the independent variable to the complete path so that we can open it as an image, and dependent variable will need to be split on the space character, so that it becomes a list.

In [ ]:
#@ Diffrent returns this time

def get_x(r):
  return path/'train'/r['fname']

def get_y(r):
  return r['labels'].split(' ')

dblock = DataBlock(get_x= get_x, get_y= get_y)
dsets = dblock.datasets(df)
dsets.train[0]

##### **4. Defining The Block Types**

---

We now need to specify the block types. Block types define how the data is processed into the correct format. For images, we use ```ImageBlock``` for the input and ```MultiCategoryBlock``` for the output, since we are doing multi-label classification.

In [ ]:
dblock = DataBlock(blocks= (ImageBlock, MultiCategoryBlock),
                   get_x= get_x, get_y= get_y)
dsets= dblock.datasets(df)
dsets.train[0]

> Note: ```TensorMultiCategory``` in output represents the multi-label format where a 1 indicates that a particular category is present for that item (similar to one-hot encoding).

##### **5. One-Hot Encoding**

---

In multi-label classification, we use one-hot encoding to represent labels.

In [ ]:
##@ We can get the actual labels using torch.where function

idxs = torch.where(dsets.train[0][1]==1.)[0]
dsets.train.vocab[idxs]

<details>
  <summary>
    <b>
      Explaining the code snippet:
    </b>
  </summary>
<ul>
  <li>
  <Code>dsets.train[0][1]</code>: Here <code>dests.train[0]</code> part gets the first training example and the <code>[1]</code> part gets the label or target associated with that example.
  </li>
  <li>
  <code>torch.where(...)[0]</code> the <code>[0]</code> here selects the first element of the tuple returned by <code>torch.where</code>, which contains the indices of <code>True</code> elements.
  </li>
</ul>
</details>

##### **6. Handling the Validation Set**

---

By default, the DataBlock uses a random split to create training and validation datasets. However, we can define our own custom split by using a ```splitter``` function. This function takes the entire ```DataFrame``` and returns two lists: one for the training set and one for the validation set.

In [ ]:
#@ Defining the custom splitter function

def splitter(df):
  train = df.index[~df['is_valid']].tolist()  #The ~ is used to indicate is False ie., is not valid
  valid = df.index[df['is_valid']].tolist()
  return train, valid

## Passing the function created to the DataBlock

dblock= DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                  splitter= splitter,
                  get_x= get_x, get_y= get_y)

dsets = dblock.datasets(df)
dsets.train[0]

##### **7. Creating DataLoaders**

---

After confirming that the individual items look correct, we now need to ensure that every item has the same size, as PyTorch requires tensors of the same shape. We can use transforms like ```RandomResizedCrop``` to resize the images.

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x,
                   get_y=get_y,
                   item_tfms=RandomResizedCrop(128, min_scale=0.35))
dls = dblock.dataloaders(df)

In [ ]:
##@ Showing the batches
dls.show_batch(nrows=1, ncols=4)

##### **8. Debugging and Validation**

---

We got the correct labels here. But, if anything goes wrong when creating our ```DataLoaders``` from our ```DataBlock```, we can use the ```summary()``` method to check the contents of the ```DataBlock``` and its transfromations

In [ ]:
dblock.summary(df)

### Binary Cross Entropy

In [ ]:
#@ Creating the Learner

learn = vision_learner(dls, resnet18)

In [ ]:
##@ Creating minibatch
x, y= to_cpu(dls.train.one_batch())
activ = learn.model(x)
activ.shape

--- Batch Size: 64 and  we need to calculate the probability of each of 20 categories. ---

In [ ]:
##@ Checking  one of the entries
activ[0]

The tensors are everywhere. So scaling them in between 0 to 1.

In [ ]:
##@ Using Sigmoid Function
def binary_cross_entropy (inputs, targets):
  inputs= inputs.sigmoid()                  ## Sigmoid activation
  return -torch.where(targets==1, inputs, 1-inputs).log().mean()

<details>
  <summary>
    <b>
      The reason behind using - 've" sign before <code>torch.where().log()</code> :
    </b>
  </summary>
  Since the logarithm of a number between 0 and 1 is negative, and  we want the loss to be positive.

  Also, cross entropy loss is also known as negative log-likelihood.

  So, we negate the log values to get the positive loss values.
</details>

<details>
  <summary>
    <b>
      Why not Softmax or NLLLoss?
    </b>
  </summary>
  <ul>
  <li>
    <b>Softmax</b> requires all predictions to sum to 1, which is not ideal for multi-label classification, where we may have multiple labels for a single image.
  </li>
  <li>
    <b>NLLLoss</b> works with single-label classification, where the target is a single class, but we need a method that handles multiple labels independently.
  </li>
  </ul>
</details>

In [ ]:
##@ Using PyTorch Builtin BCE Loss Function

loss_func = nn.BCEWithLogitsLoss()
activ_tensor = activ.as_subclass(torch.Tensor)
y_tensor = y.as_subclass(torch.Tensor)
loss = loss_func(activ_tensor, y_tensor)
loss

#### Metric For Multi-Label Classification:

For multi-label classification, we can't use the standard accuracy metric because we might have multiple labels for each input. Here's a modified version of the accuracy function that works for multi-label problems:

In [ ]:
def accuracy_multi(inp, targ, thresh= 0.5, sigmoid= True):
  if sigmoid:           #Apply sigmoid if required
    inp = inp.sigmoid()

  return ((inp > thresh)== targ.bool()).float().mean()

<details>
  <summary>
    <b>
      How it works:
    </b>
  </summary>
  <ul>
  <li>
  <b>Thresholding:</b> After applying the sigmoid to the activations, we need to decide which class activations are considered 1 (present) and which are 0 (absent). This is done by applying a threshold (default 0.5).
  </li>
  <li>
  <b>Comparing:</b> We compare the thresholded predictions to the target labels, then compute the accuracy by averaging the matches across the batch.
  </li>
  </ul>
</details>

#### Customizing the Threshold

We can modify the default threshold for the accuracy function using ```partial``` from Python’s ```functools``` to create a version of ```accuracy_multi``` with a custom threshold.

In [ ]:
##@ creating next learner with threshold

from functools import partial

#@ Setting custom threshold of 0.2
learn = vision_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.2))

In [ ]:
##@ Training the model:
"Once the loss function and metrics are set up, we can train the model as:"

learn.fine_tune(3, base_lr=3e-3, freeze_epochs=4)

In [ ]:
##@ Validating the model : On low threshold
learn.metrics = partial(accuracy_multi, thresh= 0.1)
learn.validate()

In [ ]:
##@ Validating the model : On high threshold
learn.metrics = partial(accuracy_multi, thresh= 0.99)
learn.validate()

In [ ]:
##@ Finding the best threshold value
preds, targs = learn.get_preds()
accuracy_multi(preds, targs, thresh= 0.9, sigmoid= False)

In [ ]:
###@ Now using this approach to find the best threshold level

xs = torch.linspace(0.05, 0.9575, 35)
accs= [accuracy_multi(preds, targs, thresh = i, sigmoid= False) for i in xs]
plt.plot(xs, accs)

## Regression


### Key Insights:

##### 1. Models Beyond Domains:

---


Fastai organizes applications by domains (vision, NLP, etc.), but models are actually defined by:

- Independent Variables (inputs).
- Dependent Variables (outputs).
- Loss Functions.

This flexibility allows crafting solutions for novel problems (e.g., image-to-text, text-to-image).

##### 2. **Example Problem: Image Regression**

---



**Objective:** \
Predict the center of a person's face in an image (key point detection).

**Output:** \
Two float values (row and column of the face center).

### Dataset: Biwi Kinect Head Pose


##### 1. Dataset Details

---

- Contains images (```_rgb.jpg```) and corresponding pose files (```_pose.txt```).
- Each folder represents one person; the task is to predict the center of the head.

##### 2. Dataset Downloading and Exploration:

---

In [ ]:
path = untar_data(URLs.BIWI_HEAD_POSE)
path.ls()

In [ ]:
(path/'02').ls()          ##Viewing the contents of folder '02'

##### 3. Preping Image and Pose Data

---

In [ ]:
#@ Extracting the pose files:

img_files = get_image_files(path)

def img2pose(x):
  return Path(f'{str(x)[:-7]}pose.txt')

img2pose(img_files[0])

In [ ]:
#@ Image Visualization:

im = PILImage.create(img_files[0])
im.shape

In [ ]:
im.to_thumb(160)

##### 4. Extracting the Key Points

---

In [ ]:
#@ Function to calculate the face center:

cal = np.genfromtxt(path/'01'/'rgb.cal', skip_footer=6)

def get_ctr(f):
  ctr = np.genfromtxt(img2pose(f), skip_header=3)
  c1 = ctr[0] * cal[0][0]/ctr[2] + cal[0][2]
  c2 = ctr[1] * cal[1][1]/ctr[2] + cal[1][2]
  return tensor([c1,c2])

In [ ]:
 get_ctr(img_files[0])

### DataBlock API: Flexible Data Preparation

##### 1. Custom Splitter:

---

In [ ]:
"Ensure images of the same person don't mix between training and validation"

splitter = FuncSplitter(lambda o: o.parent.name == '13')

##### 2. Data Block Definition:

---

Use ```PointBlock``` for coordinate labels, enabling augmentation on both images and coordinates

In [ ]:
biwi = DataBlock(
    blocks=(ImageBlock, PointBlock),
    get_items=get_image_files,
    get_y=get_ctr,
    splitter=splitter,
    batch_tfms=[*aug_transforms(size=(240, 320)), Normalize.from_stats(*imagenet_stats)]
)

##### 3. Data Loaders:

---

In [ ]:
dls = biwi.dataloaders(path)
dls.show_batch(max_n = 9, figsize= (8,6))

### Model Training:

##### 1. Understanding the Data Shapes:

---

In [ ]:
##@ Mini Batches Shapes
xb, yb = dls.one_batch()
xb.shape, yb.shape

In [ ]:
##! Checking the file

yb[0]

##### 2. Learner Setup

---

In [ ]:
##@ Using a pretrained ResNet Model for transfer Learning

learn = vision_learner(dls, resnet18, y_range=(-1,1))

##### 3. Loss Function

---

**Default:** Mean Squared Error (MSE)
- Predicts values close to the true coordinates.
- Alternative: Pass a custom loss function using `loss_func` in `vision_learner`.

##### 4. Finding and Setting Learning Rate

---

In [ ]:
learn.lr_find()